In [1]:
import json  
import os, sys, pathlib
import pickle

import matplotlib.pyplot as plt
import cv2

import importlib


In [3]:
os.getcwd()


'/home/lusha/star_code/notebooks'

In [ ]:
BASE_DIR = pathlib.Path.cwd().parent
sys.path.append(str(BASE_DIR / 'src'))


In [8]:
import ollama_manager as om
import main
import STAR_utils.visualization_tools.qa_visualization as qaviz


In [9]:
# please replace following data path to your local path
# all data can be download from our homepages (http://star.csail.mit.edu or https://bobbywu.com/STAR)

# root_dir = '/root/user/'
raw_video_dir = BASE_DIR / 'data/datasets/action-genome/Charades_v1_480/'
raw_frame_dir = BASE_DIR / 'data/datasets/action-genome/frames/'
annotation_dir = BASE_DIR / 'src/STAR_utils/annotations/STAR_classes/'
# pose_dir = root_dir + 'STAR/pose/'
# qa_train_dir = root_dir + 'STAR_train.json' 
# qa_val_dir = root_dir + 'STAR/STAR_val.json' 
# qa_test_dir = root_dir + 'STAR/STAR_test.json' 
save_video_dir = BASE_DIR / 'visualization_tmp'
fps = pickle.load(open(annotation_dir / 'video_fps','rb'))


In [11]:
with open(BASE_DIR / 'outputs/responses_genframes:7b_202504010_22:47:00.jsonl', 'r') as f:
    content = [json.loads(l) for l in f.readlines()]


In [14]:
len(content)


499

In [20]:
len(set(i['qid'] for i in content))


499

In [46]:
from collections import OrderedDict
class QuestionProcessor:
# ... (existing code)

    def __init__(self, stsg_filepath, stsg_buffer_size=1000):
        self.stsg_buffer_size = stsg_buffer_size  # Number of items to buffer
        self._stsg_positions = None
        self._stsg_buffer = OrderedDict()  # Maintains insertion order
        self._stsg_file_handle = None
        self.stsg_file_path = stsg_filepath

    def _build_stsg_index(self, file_path):
        self._stsg_index = {}
        id_key = None
        first_line = True
        try:
            with open(file_path, 'rb') as f:
                while True:
                    byte_offset = f.tell()
                    line = f.readline()

                    if not line:
                        break

                    try:
                        item = json.loads(line)
                        
                        if first_line:
                            if "qid" in item:
                                id_key = "qid"
                            elif "question_id" in item:
                                id_key = "question_id"
                            else:
                                raise ValueError("STSG file lines must contain either 'qid' or 'question_id'")
                            first_line = False

                        if id_key not in item:
                            print(f"Warning: Line at offset {byte_offset} missing ID key '{id_key}'. Skipping.")
                            continue

                        self._stsg_index[item[id_key]] = byte_offset

                    except json.JSONDecodeError:
                        print(f"Warning: Unable to decode line at offset {byte_offset}. Skipping.")
                        continue
        except FileNotFoundError:
            raise FileNotFoundError(f"STSG file not found during indexing: {file_path}")
        except IOError as e:
            raise IOError(f"Error reading STSG file during indexing: {e}") from e





In [47]:
a = QuestionProcessor(BASE_DIR / 'outputs/responses_genframes:7b_202504010_22:47:00.jsonl')
a._build_stsg_index(BASE_DIR / 'outputs/responses_genframes:7b_202504010_22:47:00.jsonl')
a._stsg_index.items()


dict_items([('Interaction_T1_13', 0), ('Interaction_T1_14', 10603), ('Interaction_T1_31', 22263), ('Interaction_T1_32', 28935), ('Interaction_T1_40', 38005), ('Interaction_T1_43', 49271), ('Interaction_T1_70', 58658), ('Interaction_T1_71', 67380), ('Interaction_T1_76', 76561), ('Interaction_T1_78', 85861), ('Interaction_T1_81', 93153), ('Interaction_T1_95', 105532), ('Interaction_T1_100', 113686), ('Interaction_T1_101', 122374), ('Interaction_T1_104', 130376), ('Interaction_T1_106', 138799), ('Interaction_T1_111', 146039), ('Interaction_T1_137', 153754), ('Interaction_T1_139', 164539), ('Interaction_T1_147', 175372), ('Interaction_T1_148', 185465), ('Interaction_T1_171', 192817), ('Interaction_T1_172', 199928), ('Interaction_T1_173', 205893), ('Interaction_T1_184', 220278), ('Interaction_T1_188', 230379), ('Interaction_T1_197', 245078), ('Interaction_T1_198', 253392), ('Interaction_T1_210', 259810), ('Interaction_T1_211', 270626), ('Interaction_T1_212', 280545), ('Interaction_T1_213', 

In [ ]:
sorted_positions = sorted(load_positions.items(), key=lambda x: x[1])
